In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from peft import PeftModel, PeftConfig
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-1000"
config = PeftConfig.from_pretrained(peft_model_id)


In [3]:
inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:25<00:00,  3.20s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True
)

In [5]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [2]:
# tokenizer = AutoTokenizer.from_pretrained(
#     "/home/ysx/models/internlm-chat-7b",
#     trust_remote_code=True
# )

In [3]:
# model = AutoModelForCausalLM.from_pretrained(
#     "/home/ysx/models/internlm-chat-7b",
#     load_in_4bit=True,
#     device_map="auto",
#     trust_remote_code=True
# )

Loading checkpoint shards: 100%|██████████| 8/8 [00:24<00:00,  3.11s/it]


In [6]:
model = model.eval()


In [9]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### instruction:\n{instruction}\n\n### input:\n{input}\n\n### responds:\n"
    )

In [10]:
text = generate_prompt("请回答下面问题", "五行是什么")

In [15]:
text = "阴阳五行理论的核心是什么？"

In [16]:
response, history = model.chat(tokenizer, text, history=[])
print(response)


阴阳五行理论的核心是五行和阴阳的相互关系。五行，即木、火、土、金、水，是宇宙万物的构成要素，阴阳是万物的基本属性。五行之间，木生火，火生土，土生金，金生水，水生木，木生土生金，土生金生水，水生木生火，火生土生金，金生水生木，木生火生土，这是五行之间的生克关系。五行相生，即：木生火，火生土，土生金，金生水，水生木；五行相克，即：木克土，土克水，水克火，火克金，金克木。五行之间，在自然界中，是相互作用，相生相克，相生即相互帮助，相克即相互制约。在人体内部，也是相互作用的。五行之间，在自然界中，是相互作用，相生相克，相生即相互帮助，相克即相互制约。在人体内部，也是相互作用的。五行之间，在人体内部，是相互作用的，在中医学中，用五行理论，来解释人体内部的生理和病理现象。人体内部的五行，即木、火、土、金、水。木主肝，火主心，土主脾，金主肺，水主肾。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通于肝，木火相生，木火相克，木火相克，木火相生。木火通于心，木火相生，木火相克，木火相克，木火相生。木火通于肾，木火相生，木火相克，木火相克，木火相生。木火通